# CH06-TORCH.AUTOGRAD

官方链接：https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html#


在训练神经网络时，最常用的算法是 反向传播（back propagation） 算法，在该算法中，参数（model weights）根据损失函数的梯度进行调整。

为了计算这些梯度，PyTorch中有一个名为`torch.autograd()` 的内置微分引擎，它支持任何计算图的梯度计算。

考虑最简单的一层神经网络：具有输入x、参数w和b，以及损失函数，它可以通过以下方式定义：

In [2]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## 1 张量，Function类和计算图

上述代码，计算图如下：

![img](./data_ch06/compute-graph.jpg)


其中，w和b是模型参数，是需要我们优化的参数。

为了能计算出这些变量关于损失的梯度，需要设置tensor的`requires_grad`属性：

+ requires_grad可以在创建tensor的时候使用

+ 也可以通过`x.requires_grad_(True)`后设置

应用在张量上的，用来构建计算图的函数，实际上是类`Function`的对象。该对象知道如何前向计算、如何在反向传播中计算其导数。

对反向传播函数的引用存储在tensor属性`.grad_fn`中：

In [4]:
print('Gradient function for z =',z.grad_fn)

print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x000002DEF469AF40>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x000002DEF469A6D0>


## 6.2 计算梯度

为了优化神经网络中的权重参数，我们需要计算出损失函数对参数的导数。即固定值x和y下的

$$
\frac{\partial loss}{\partial w} \; and \; \frac{\partial loss}{\partial b}
$$

使用`loss.backward()`，获得参数w和b的梯度：

In [5]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2406, 0.2332, 0.1253],
        [0.2406, 0.2332, 0.1253],
        [0.2406, 0.2332, 0.1253],
        [0.2406, 0.2332, 0.1253],
        [0.2406, 0.2332, 0.1253]])
tensor([0.2406, 0.2332, 0.1253])


Note:

+ 只能获取计算图中叶子节点的梯度属性(这些叶子节点的requires_grad属性为True)，对于计算图中的其他节点，梯度都不可用。

+ backward出于性能原因，在给定的计算图上只能使用一次梯度计算。如果需要在同一个计算图上多次调用backward，我们需要传递 retain_graph=True 给backward

## 6.3 关闭梯度跟踪

默认情况下，所有张量的属性`requires_grad=True`，即默认跟踪它们的计算历史并支持梯度计算。

但是，有些情况下我们不需要跟踪梯度，比如在验证集上检验模型性能的时候，即我们只想通过网络进行前向计算。这个时候，我们可以通过调用`torch.no_grad()`块包围我们的计算代码来停止跟踪计算:

In [7]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b   
print(z.requires_grad)

True
False


关闭梯度跟踪的原因有：

+ 将神经网络中的某些参数冻结，该场景常用于微调预训练模型

微调预训练模型:https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

+ 仅需要前向计算的时候，加快计算速度，常用于测试模型在验证集上的表现

从概念上讲，autograd 在由Function 对象组成的有向无环图 (DAG) 中保存数据（张量）和所有已执行操作（以及生成的新张量）的记录 。在这个 DAG 中，叶子是输入张量，根是输出张量。通过从根到叶跟踪此计算图，可以使用链式法则自动计算梯度。

在前向传递中，autograd 同时做两件事：

+ 计算结果张量

+ 维护在 DAG 中的梯度函数。

当`.backward()`在 DAG 的根节点上调用时，反向传递开始。它会：

+ 计算来自每个`.grad_fn`的梯度

+ 将它们累积在各自张量的`.grad`属性中

+ 使用链式法则，一直传播到叶子张量。


Note:

PyTorch 中,DAG 是动态的。需要注意的是，该图是从头开始重新创建的；每次 .backward()调用后，autograd 开始填充新图形。